In [1]:
from SMT_model import *
from time import time as time_clock

In [2]:
#Demo with instance 01
Path = ('../Instances/inst01.dat')

timeout = 300


In [3]:
#Read the file from the path
with open(Path, 'r') as file:
    data = file.read()

instance = data_parsing(data)
print(instance)

{'m': 2, 'n': 6, 'l': [15, 10], 's': [3, 2, 6, 5, 4, 4], 'D': [[0, 3, 4, 5, 6, 6, 2], [3, 0, 1, 4, 5, 7, 3], [4, 1, 0, 5, 6, 6, 4], [4, 4, 5, 0, 3, 3, 2], [6, 7, 8, 3, 0, 2, 4], [6, 7, 8, 3, 2, 0, 4], [2, 3, 4, 3, 4, 4, 0]]}


In [4]:
def run_smt(_, instance, timeout, sb=False):
    generation_start_time = time_clock()

    o, x, max_distance = stm_model(instance, timeout, sb)
    generation_duration = time_clock() - generation_start_time
    o.set("timeout", int(timeout - generation_duration) * 1000)

    obj = o.minimize(max_distance)
    res = o.check()
    if res in [sat, unknown]:
        result_formatted = format_solution(instance, o.model(), x)
        return result_formatted, res==sat
    elif res == unsat:
        return "unsat"
    else:
        return "unknown"

In [5]:
print(run_smt(_, instance, timeout, True))

[distances = Store(Store(Store(Store(Store(Store(K(Int,
                                        Store(Store(Store(Store(K(Int,
                                        4),
                                        6,
                                        0),
                                        1,
                                        3),
                                        3,
                                        3),
                                        0,
                                        2)),
                                        1,
                                        Store(Store(Store(Store(Store(K(Int,
                                        3),
                                        1,
                                        0),
                                        3,
                                        4),
                                        4,
                                        5),
                                        2,
            

x_0_5 = 6,
 x_1_3 = 5,
 max_distance = 14,
 x_1_0 = 6,
 y_1 = 14,
 load_1 = 10,
 y_0 = 13,
 x_0_0 = 6,
 load_0 = 14,
 x_0_6 = 6,
 x_1_6 = 6]
([[1, 3, 4], [2, 5, 6]], True)